In [27]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time


In [26]:
url = 'https://www.melon.com/chart/index.htm'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

result = requests.get(url, headers=header)

if result.status_code != 200:
    print('result.status_code = ', result.status_code)

soup = BeautifulSoup(result.text, 'html.parser')
trs = soup.select(' tbody > tr')

lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    img = tr.select_one('img')['src']
    title = tr.select_one('.rank01 > span > a').get_text()
    artist = tr.select_one('.rank02 > a').get_text()
    album = tr.select_one('.rank03 > a').get_text()
    lines.append({'순위': rank, '곡명': title, '아티스트': artist, '앨범': album, '이미지': img})

df = pd.DataFrame(lines)     
df.head()

,순위,곡명,아티스트,앨범,이미지
0,1,I AM,IVE (아이브),I've IVE,https://cdnimg.melon.co.kr/cm2/album/images/11...
1,2,UNFORGIVEN (feat. Nile Rodgers),LE SSERAFIM (르세라핌),UNFORGIVEN,https://cdnimg.melon.co.kr/cm2/album/images/11...
2,3,Kitsch,IVE (아이브),I've IVE,https://cdnimg.melon.co.kr/cm2/album/images/11...
3,4,손오공,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album ‘FML',https://cdnimg.melon.co.kr/cm2/album/images/11...
4,5,Spicy,aespa,MY WORLD - The 3rd Mini Album,https://cdnimg.melon.co.kr/cm2/album/images/11...


In [25]:
now = datetime.now()

df.to_csv(f"static/data/멜론_{now.strftime('%Y')[2:]}{now.strftime('%m%d%H')}.csv", index=False)



,순위,곡명,아티스트,앨범,이미지
0,1,I AM,IVE (아이브),I've IVE,https://cdnimg.melon.co.kr/cm2/album/images/11...
1,2,UNFORGIVEN (feat. Nile Rodgers),LE SSERAFIM (르세라핌),UNFORGIVEN,https://cdnimg.melon.co.kr/cm2/album/images/11...
2,3,Kitsch,IVE (아이브),I've IVE,https://cdnimg.melon.co.kr/cm2/album/images/11...
3,4,손오공,세븐틴 (SEVENTEEN),SEVENTEEN 10th Mini Album ‘FML',https://cdnimg.melon.co.kr/cm2/album/images/11...
4,5,Spicy,aespa,MY WORLD - The 3rd Mini Album,https://cdnimg.melon.co.kr/cm2/album/images/11...


In [87]:
import re
def convert_num(s):
    return int(re.sub('[,억개 ]', '', s).replace('만', '0000'))

In [92]:
# webdrive로

base_url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')

lines = []
for i in range(1, 101):
    driver.get(base_url + str(i))
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one(' h1 > a').get_text().strip()

        subscriber = convert_num(tr.select_one('.subscriber_cnt').get_text())
        view = convert_num(tr.select_one('.view_cnt').get_text())
        video = convert_num(tr.select_one('.video_cnt').get_text())

        lines.append({'순위':rank, '카테고리':category, '채널': channel, '구독자수': subscriber, '조회수': view, '비디오': video})


df = pd.DataFrame(lines)

driver.close()

df.head()

100
BLACKPINK
BANGTANTV
HYBE LABELS
SMTOWN
JYP Entertainment
Boram Tube Vlog [보람튜브 브이로그]
1MILLION Dance Studio
1theK (원더케이)
Alan Becker
Mnet K-POP


,순위,카테고리,채널,구독자수,조회수,비디오
0,1,음악/댄스/가수,BLACKPINK,88000000,30959130000,514
1,2,음악/댄스/가수,BANGTANTV,74600000,20279390000,2195
2,3,음악/댄스/가수,HYBE LABELS,70600000,27690310000,1185
3,4,음악/댄스/가수,SMTOWN,31700000,26928580000,4121
4,5,음악/댄스/가수,JYP Entertainment,26900000,19395230000,1678


In [90]:

base_url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='
# driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')


lines = []
for i in range(1, 2):
    # driver.get(base_url + str(i))
    # time.sleep(3)
    result = requests.get(base_url + str(i), headers=header)


    soup = BeautifulSoup(result.text, 'html.parser')
    trs = soup.select('.aos-init')
    print(len(trs))
    for tr in trs[:10]:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = ''
        # channel = tr.select('.subject > h1')
        # channel = tr.select('.subject > h1').get_text().strip() ## 읽혀지지 않는다. 
        # print(type(channel[0].get_attribute_list()))

        subscriber = convert_num(tr.select_one('.subscriber_cnt').get_text())
        view = convert_num(tr.select_one('.view_cnt').get_text())
        video = convert_num(tr.select_one('.video_cnt').get_text())

        lines.append({'순위':rank, '카테고리':category, '채널': channel, '구독자수': subscriber, '조회수': view, '비디오': video})


df = pd.DataFrame(lines)

driver.close()

df.head()

0


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=112.0.5615.140)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0099DCE3+50899]
	(No symbol) [0x0092E111]
	(No symbol) [0x00835588]
	(No symbol) [0x008270CB]
	(No symbol) [0x0082778A]
	(No symbol) [0x00827728]
	(No symbol) [0x0081C37A]
	(No symbol) [0x0081CBD7]
	(No symbol) [0x00883C48]
	(No symbol) [0x0087B8C3]
	(No symbol) [0x00857708]
	(No symbol) [0x0085886D]
	GetHandleVerifier [0x00C03EAE+2566302]
	GetHandleVerifier [0x00C392B1+2784417]
	GetHandleVerifier [0x00C3327C+2759788]
	GetHandleVerifier [0x00A35740+672048]
	(No symbol) [0x00938872]
	(No symbol) [0x009341C8]
	(No symbol) [0x009342AB]
	(No symbol) [0x009271B7]
	BaseThreadInitThunk [0x775E00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77CC7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77CC7B1E+238]


In [49]:
trs = driver.find_elements(By.CLASS_NAME, 'aos-init')
len(trs)

100

In [59]:
tr = trs[0]
int(tr.find_element(By.CLASS_NAME, 'rank').text.strip())

1

In [51]:
tr.find_element(By.CLASS_NAME, 'category').text.strip()

'[음악/댄스/가수]'

In [52]:
tr.find_element(By.CSS_SELECTOR, 'td > h1 > a').text.strip()

'BLACKPINK'

In [53]:
tr.find_element(By.CLASS_NAME, 'subscriber_cnt').text.strip()

'8800만'

In [54]:
tr.find_element(By.CLASS_NAME, 'view_cnt').text.strip()

'309억5913만'

In [55]:

tr.find_element(By.CLASS_NAME, 'video_cnt').text.strip()

'514개'

In [82]:
base_url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')

lines = []
for i in range(1, 2):
    driver.get(base_url + str(i))
    time.sleep(3)

    trs = driver.find_elements(By.CLASS_NAME, 'aos-init')
    print(len(trs))
    
    for tr in trs[:10]:
        
        rank = tr.find_element(By.CLASS_NAME, 'rank').text.strip()
        print(rank)
       
        # category = tr.find_element(By.CSS_SELECTOR, '.category').text.strip()[1:-1]
        # channel = tr.find_element(By.CSS_SELECTOR, 'td > h1 > a').text.strip()

        # subscriber = convert_num(tr.find_element(By.CSS_SELECTOR, '.subscriber_cnt').text.strip())
        # view = convert_num(tr.find_element(By.CSS_SELECTOR, '.view_cnt').text.strip())
        # video = convert_num(tr.find_element(By.CSS_SELECTOR, '.video_cnt').text.strip())

        # # print(rank, subscriber, view,  video)

        # lines.append({'순위':rank, '카테고리':category, '채널': channel, '구독자수': subscriber, '조회수': view, '비디오': video})


df = pd.DataFrame(lines)

driver.close()

df.head()


100
1
2










""


1